# Part 3: Fibonacci Example
# Chapter 5: Padding

In [2]:
from dsl import Circuit, StepType
from cb import eq
from util import F
from chiquito_ast import Last

class FiboFirstStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a, 1))
        self.constr(eq(self.circuit.b, 1))
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class Padding(StepType):
    def setup(self):
        self.transition(eq(self.circuit.b, self.circuit.b.next()))

    def wg(self, b_value):
        self.assign(self.circuit.b, F(b_value))


class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")
        
        self.fibo_first_step = self.step_type(FiboFirstStep(self, "fibo_first_step"))
        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        self.padding = self.step_type(Padding(self, "padding"))

        self.pragma_num_steps(100)
        self.pragma_first_step(self.fibo_first_step)
        self.pragma_last_step(self.padding)

        self.expose(self.a, Last())
        
    def trace(self, n):
        self.add(self.fibo_first_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, n):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a
        while(self.needs_padding()):
            self.add(self.padding, prev_a)

fibo = Fibonacci()
fibo_witness = fibo.gen_witness(30)
fibo.halo2_mock_prover(fibo_witness)

there's last step
100
1
243437734546730727453508299613593537034
Err(
    [
        ConstraintCaseDebug {
            constraint: Constraint {
                gate: Gate {
                    index: 0,
                    name: "main",
                },
                index: 1,
                name: "fibo_step::(b == next(a)) => (b + -next(a)) => Product(Fixed { query_index: 0, column_index: 0, rotation: Rotation(0) }, Product(Sum(Constant(0x0000000000000000000000000000000000000000000000000000000000000001), Negated(Fixed { query_index: 1, column_index: 2, rotation: Rotation(0) })), Product(Advice { query_index: 0, column_index: 3, rotation: Rotation(0) }, Sum(Advice { query_index: 2, column_index: 1, rotation: Rotation(0) }, Negated(Advice { query_index: 4, column_index: 0, rotation: Rotation(1) })))))",
            },
            location: InRegion {
                region: Region 0 ('circuit'),
                offset: 29,
            },
            cell_values: [
                (
 